In [43]:
nodes= [:P, :W, :C, :R, :O, :F, :M]
supply = [:P, :W]
plant = [:C, :R]
city = [:O, :F, :M]
arcs=[(:P,:C), (:P,:R), (:W,:C), (:W,:R), 
      (:C,:O), (:C,:F), (:C,:M), 
      (:R,:O), (:R,:F), (:R,:M)]

R = Dict(zip(city, [3.5, 2.5, 2.85]))
B = Dict(zip(nodes, [400, 650, 0, 0, -550, -600, -700]))
C = Dict(zip(arcs, [0.35, 0.5, 0.3, 0.3, 0.6, 0.5, 0.25, 0.25, 0.25, 0.65]))
P = Dict(zip(plant, [0.25, 0.3]))
H = Dict(zip(plant, [900, 500]))
M = Dict(zip(city, [550, 600, 700]))

using JuMP, Cbc

m = Model()

@variable(m, x[arcs] >= 0) # amount of transported from node i to j
@variable(m, y[plant] >= 0) # amount of processed at plant 
@variable(m, z[city] >= 0) # amount of milk sold at city

@objective(m, Max, sum(R[i] * z[i] for i in city) - sum(C[i] * x[i] for i in arcs) - sum(P[i] * y[i] for i in plant))

@constraint(m, flowbalance[i in nodes], 
sum(x[(i,j)] for j in nodes if (i,j) in arcs) - sum(x[(j,i)] for j in nodes if (j,i) in arcs) == B[i])

@constraint(m, max_process[i in plant], y[i] <= H[i])

@constraint(m, max_sold[i in city], z[i] <= M[i])

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

Presolve 2 (-10) rows, 5 (-10) columns and 10 (-15) elements
0  Obj 4282.5 Primal inf 800 (1) Dual inf 0.549998 (2)
1  Obj 4537.5 Primal inf 800 (1)
Primal infeasible - objective value 4537.5
Presolved problem not optimal, resolve after postsolve
After Postsolve, objective 4537.5, infeasibilities - dual 0 (0), primal 800 (1)
PrimalInfeasible objective 4537.5 - 1 iterations time 0.002, Presolve 0.00


LoadError: MethodError: [0mCannot `convert` an object of type [92mMissing[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number at C:\Users\peren\AppData\Local\Programs\Julia-1.7.1\share\julia\base\number.jl:6
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number at C:\Users\peren\AppData\Local\Programs\Julia-1.7.1\share\julia\base\number.jl:7
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at C:\Users\peren\AppData\Local\Programs\Julia-1.7.1\share\julia\base\twiceprecision.jl:262
[0m  ...

In [18]:
using JuMP, Cbc

m = Model()

@variable(m, yo >= 0)
@variable(m, yf >= 0)
@variable(m, ym >= 0)
@variable(m, x[arcs] >= 0)

@objective(m, Max, 3.5yo + 2.5yf + 2.85ym - (0.35x[(:P, :C)] + 0.5x[(:P, :R)] + 0.3x[(:W, :C)] + 0.3x[(:W, :R)] +
        0.6x[(:C, :O)] + 0.5x[(:C, :F)] + 0.25x[(:C, :M)] + 0.25x[(:R, :O)] + 0.25x[(:R, :F)] + 0.65x[(:R, :M)] ))

@constraint(m, limit_P, x[(:P,:C)] + x[(:P,:R)] <= 400)
@constraint(m, limit_W, x[(:W,:C)] + x[(:W,:R)] <= 650)
@constraint(m, balance_C, -x[(:P,:C)] - x[(:W,:C)] + x[(:C,:O)] + x[(:C,:F)] + x[(:C,:M)] == 0)
@constraint(m, balance_R, -x[(:P,:R)] - x[(:W,:R)] + x[(:R,:O)] + x[(:R,:F)] + x[(:R,:M)] == 0)
@constraint(m, limit_C, x[(:C,:O)] + x[(:C,:F)] + x[(:C,:M)] <= 900)
@constraint(m, limit_R, x[(:R,:O)] + x[(:R,:F)] + x[(:R,:M)] <= 500)
@constraint(m, limit_O, -x[(:C,:O)] - x[(:R,:O)] <= -550)
@constraint(m, limit_F, -x[(:C,:F)] - x[(:R,:F)] <= -600)
@constraint(m, limit_M, -x[(:C,:M)] - x[(:R,:M)] <= -700)
@constraint(m, balance_all, x[(:P,:C)] + x[(:P,:R)] + x[(:W,:C)] + x[(:W,:R)] -x[(:C,:O)] - x[(:R,:O)] -x[(:C,:F)] - x[(:R,:F)]
-x[(:C,:M)] - x[(:R,:M)] == 0)

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

Presolve thinks problem is unbounded
Analysis indicates model infeasible or unbounded
0  Obj -0 Primal inf 1850 (3) Dual inf 8.8499997 (3)
7  Obj 8.8499999e+10 Primal inf 1300 (3)
8  Obj 8.8499999e+10 Primal inf 1250 (2)
8  Obj 8.8499999e+10 Primal inf 1250 (2)
8  Obj 8.8499999e+10 Primal inf 1250 (2)
8  Obj 4.425e+11 Primal inf 1250 (2)
8  Obj 2.2125e+12 Primal inf 1250 (2)
8  Obj 1.10625e+13 Primal inf 1250 (2)
8  Obj 5.53125e+13 Primal inf 1250 (2)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual inf 8.8499997 (3)
8  Obj -1245 Primal inf 1250 (2) Dual in

LoadError: MethodError: [0mCannot `convert` an object of type [92mMissing[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number at C:\Users\peren\AppData\Local\Programs\Julia-1.7.1\share\julia\base\number.jl:6
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number at C:\Users\peren\AppData\Local\Programs\Julia-1.7.1\share\julia\base\number.jl:7
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at C:\Users\peren\AppData\Local\Programs\Julia-1.7.1\share\julia\base\twiceprecision.jl:262
[0m  ...